In [1]:
import pandas as pd

iris = pd.read_csv("../iris.data", header=None, names=["sepal_length", "sepal_width", 
                                                    "petal_length", "petal_width", "label"])
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,label
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

X = iris.drop(['label'], axis=1)
y = iris.label

# encode label
y_data = pd.get_dummies(y)

In [3]:
import torch
from torch.autograd import Variable
import torch.nn.functional as F

X = Variable(torch.from_numpy(X.values)).float()
y = Variable(torch.from_numpy(y_data.values)).float()

In [5]:
class Model(torch.nn.Module):
    """
    Input  layer  4
    Hidden layer1 8
    Hidden layer2 12
    Hidden layer3 8
    Output layer  3
    """
    def __init__(self):
        super(Model, self).__init__()
        self.layer1 = torch.nn.Linear(4, 8)
        self.layer2 = torch.nn.Linear(8, 12)
        self.layer3 = torch.nn.Linear(12, 8)
        self.layer4 = torch.nn.Linear(8, 3)
        
        self.sigmoid = torch.nn.Sigmoid()
        
    def forward(self, x):
        output1 = F.relu(self.layer1(x))
        output2 = F.relu(self.layer2(output1))
        output3 = F.relu(self.layer3(output2))
        y_pred  = (self.layer4(output3))
        return y_pred

In [6]:
model = Model()

In [7]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.7)

In [8]:
epochs = 10
for epoch in range(epochs):
    y_pred = model(X)
    
    loss = criterion(y_pred, y)
    print(f"Epoch: {epoch+1}\t Loss: {loss.data.item()}")
    
    # zero gradients, perform a backward pass, and update weights
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

Epoch: 1	 Loss: 0.4424777925014496
Epoch: 2	 Loss: 0.3940688371658325
Epoch: 3	 Loss: 0.3368259072303772
Epoch: 4	 Loss: 0.2842331826686859
Epoch: 5	 Loss: 0.24166730046272278
Epoch: 6	 Loss: 0.2122112661600113
Epoch: 7	 Loss: 0.1950383186340332
Epoch: 8	 Loss: 0.18648849427700043
Epoch: 9	 Loss: 0.18246619403362274
Epoch: 10	 Loss: 0.17824934422969818


In [9]:
torch.argmax(y_pred, 1)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 1,
        2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2])

Accuracy

In [10]:
import numpy as np
sum(torch.argmax(y_pred, 1).numpy() == np.argmax(y_data.values, 1)) / y_data.shape[0]

0.9466666666666667

**Compare `cpp` implementation to `torch` solution**

In [16]:
%%!
cd ..
file=$(pwd)"/iris.data"
bin/neuralnet $file 20 2 0.1

['Epoch: 1/20 complete\tTest loss: 0.135927',
 'Epoch: 2/20 complete\tTest loss: 0.13176',
 'Epoch: 3/20 complete\tTest loss: 0.1281',
 'Epoch: 4/20 complete\tTest loss: 0.121756',
 'Epoch: 5/20 complete\tTest loss: 0.118432',
 'Epoch: 6/20 complete\tTest loss: 0.115217',
 'Epoch: 7/20 complete\tTest loss: 0.111112',
 'Epoch: 8/20 complete\tTest loss: 0.10537',
 'Epoch: 9/20 complete\tTest loss: 0.102704',
 'Epoch: 10/20 complete\tTest loss: 0.100482',
 'Epoch: 11/20 complete\tTest loss: 0.0982911',
 'Epoch: 12/20 complete\tTest loss: 0.0945849',
 'Epoch: 13/20 complete\tTest loss: 0.094575',
 'Epoch: 14/20 complete\tTest loss: 0.0926059',
 'Epoch: 15/20 complete\tTest loss: 0.091264',
 'Epoch: 16/20 complete\tTest loss: 0.0890861',
 'Epoch: 17/20 complete\tTest loss: 0.0872235',
 'Epoch: 18/20 complete\tTest loss: 0.0853142',
 'Epoch: 19/20 complete\tTest loss: 0.0846057',
 'Epoch: 20/20 complete\tTest loss: 0.0833902',
 '',
 'Accuracy: 0.8',
 '',
 'Confunsion matrix',
 '10 0 0 ',
 '0